<a href="https://colab.research.google.com/github/jialun1221/scRNA-seq/blob/main/SUpport_Vector_Machine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install scanpy
import numpy as np
import pandas as pd
import scanpy as sc
import sys
import random
from sklearn import preprocessing 
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import importlib
required_libraries = ['torch', 'torchvision', 'PIL', 'matplotlib', 
                      'numpy', 'pandas']
for lib in required_libraries:
    if importlib.util.find_spec(lib) is None:
        print("%s unavailable" % lib)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.0/103.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 141.1 kB/s eta 0:00:00
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82830 sha256=0ad5b7da63e30f27fc1a645e88781eea8a83ac5ebea2dc76f159cc2b2c111b2e
  Stored in directory: /root/.cache/pip/wheels/a0/e8/c6/a37ea663620bd5200ea1ba0907ab3c217042c1d035ef606acc
  Created wheel for session-info: filename=session_info-1.0.0-py3-none-any.whl size=8042 sha256=0f3e03ff41fb3054e

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data import TensorDataset

In [ ]:
#0. Model methods import
import seaborn as sns
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn import metrics
import sklearn.metrics as metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn import svm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

adata_m1 = sc.read_h5ad("drive/MyDrive/scRNA ML classifier/data_objects_May_2022/PC_all_genes_qc.h5ad")
adata_m2 = sc.read_h5ad("drive/MyDrive/scRNA ML classifier/data_objects_May_2022/PC_HVGs_qc.h5ad")
adata_m3 = sc.read_h5ad("drive/MyDrive/scRNA ML classifier/data_objects_May_2022/no_PC_HVGs_qc.h5ad")

data_list = [adata_m1, adata_m2, adata_m3]

Mounted at /content/drive


In [ ]:
df_astro_SVM = pd.DataFrame(columns = ['Seed', 'aSVM m1 val', 'aSVM m2 val', 'aSVM m3 val','aLR SVM test', 'aSVM m2 test', 'aSVM m3 test'])

In [ ]:
dict_list_astro_SVM_m1 = []
dict_list_astro_SVM_m2 = []
dict_list_astro_SVM_m3 = []

In [ ]:
def SVMTest():
  use_gpu = torch.cuda.is_available()
  device = torch.device("cuda:0" if use_gpu else "cpu") 
  clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  #eval
  test_acc = metrics.accuracy_score(y_test, y_pred)
  print(test_acc)
  return test_acc

In [ ]:
def SVMValidation():
  use_gpu = torch.cuda.is_available()
  device = torch.device("cuda:0" if use_gpu else "cpu") 
  clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
  y_pred_val = clf.predict(X_val)
  #eval
  val_acc = metrics.accuracy_score(y_val, y_pred_val)
  print(val_acc)
  return val_acc

In [ ]:
import copy
i = 0 #method number
seed_list = []

for dataobj in data_list:
  print(i)
  print(dataobj)
  i = i + 1
  adata = dataobj

  #1. Pre-process data
  if 'level_0' not in adata.obs.columns:
    adata.obs = adata.obs.reset_index() #reset the index for X, so the column number starts from 0

  if 'X_pca' in adata.obsm:
    X = adata.obsm['X_pca'].X
  else:
    X = adata.X 
  
  y = adata.obs['disease__ontology_label'].replace({"normal": "0", "Parkinson disease": "1"})

  #2. Train-test split: 60/20/20. Random_state is the seed.Validation used for hyperparameter tuning. 
  #   Parameters for the highest accuracy can be used for test set.
  
  X_train_global, X_test_global, y_train_global, y_test_global = train_test_split(X, y, test_size=0.2, random_state=0) #test is always the same

  #3. run 10 iterations
  for m in range (10):
    X_test = copy.deepcopy(X_test_global)
    y_test = copy.deepcopy(y_test_global)
    
    if len(seed_list) < 10:
      seed = random.randint(0, 9999)
      seed_list.append(seed)
      print("seed: ", seed)
      X_train, X_val, y_train, y_val = train_test_split(X_train_global, y_train_global, test_size=0.25, random_state= seed_list[m])
    else:
      print("seed = ", seed)
      print(seed_list[m])
      X_train, X_val, y_train, y_val = train_test_split(X_train_global, y_train_global, test_size=0.25, random_state= seed_list[m])

    val_acc = SVMValidation()
    test_acc = SVMTest()

    if i == 1: #write in method 1 section
      row_dict_m1 = {'Seed': seed_list[m], 'aSVM m1 val': val_acc, 'aSVM m1 test': test_acc}
      dict_list_astro_SVM_m1.append(row_dict_m1)
    elif i == 2:
      row_dict_m2 = {'Seed': seed_list[m], 'aSVM m2 val': val_acc, 'aSVM m2 test': test_acc}
      dict_list_astro_SVM_m2.append(row_dict_m2)
    elif i == 3:
      row_dict_m3 = {'Seed': seed_list[m], 'aSVM m3 val': val_acc, 'aSVM m3 test': test_acc}
      dict_list_astro_SVM_m3.append(row_dict_m3)
      
print(dict_list_astro_SVM_m1)
print(dict_list_astro_SVM_m2)
print(dict_list_astro_SVM_m3)


0
AnnData object with n_obs × n_vars = 25019 × 30738
    obs: 'level_0', 'index', 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'Cell_Subtype', 'Cell_Type', 'disease__ontology_label', 'organ__ontology_label', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt'
    var: 'features', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'hvg', 'log1p', 'pca'
    obsm: 'X_pca'
    varm: 'PCs'
seed:  6505
0.9854116706634692
0.9842126298960832
seed:  9650
0.9884092725819345
0.9836131095123901
seed:  6588
0.9866107114308553
0.9834132693844924
seed:  9724
0.9856115107913669
0.9840127897681854
seed:  951
0.9848121502797762
0.9844124700239808
seed:  2054
0.9862110311750599
0.9848121502797762
seed:  9193
0.9836131095123901
0.9842126298960832
seed:  2525
0.9872102318145484
0.9840127897681854
seed:  4782
0.9866107114308553
0.9850119904076738
seed

In [ ]:
df_astro_m1 = pd.DataFrame.from_dict(dict_list_astro_SVM_m1)
df_astro_m2 = pd.DataFrame.from_dict(dict_list_astro_SVM_m2)
df_astro_m3 = pd.DataFrame.from_dict(dict_list_astro_SVM_m3)
f = pd.concat([df_astro_m1, df_astro_m2,df_astro_m3])
f

,Seed,aSVM m1 val,aSVM m1 test,aSVM m2 val,aSVM m2 test,aSVM m3 val,aSVM m3 test
0,6505,0.985412,0.984213,NaN,NaN,NaN,NaN
1,9650,0.988409,0.983613,NaN,NaN,NaN,NaN
2,6588,0.986611,0.983413,NaN,NaN,NaN,NaN
3,9724,0.985612,0.984013,NaN,NaN,NaN,NaN
4,951,0.984812,0.984412,NaN,NaN,NaN,NaN
5,2054,0.986211,0.984812,NaN,NaN,NaN,NaN
6,9193,0.983613,0.984213,NaN,NaN,NaN,NaN
7,2525,0.987210,0.984013,NaN,NaN,NaN,NaN
8,4782,0.986611,0.985012,NaN,NaN,NaN,NaN
9,9003,0.984412,0.984213,NaN,NaN,NaN,NaN
